In [5]:
import nibabel as nib
import numpy as np
from scipy.ndimage import label
import os
import csv
from pathlib import Path

In [6]:
def separate_clusters(tmap_path, output_dir, threshold, extent_threshold=10):
    """
    Separates non-contiguous clusters in a T-map into individual NIfTI masks,
    applying a voxel count threshold (extent threshold).

    Parameters:
        tmap_path (str): Path to the input T-map NIfTI file.
        output_dir (str): Directory to save the output NIfTI masks.
        threshold (float): Threshold value to apply to the T-map.
        extent_threshold (int): Minimum number of voxels for a cluster to be retained.

    Returns:
        tuple: (list of cluster paths, affine transformation matrix)
    """
    # Load the T-map
    tmap_img = nib.load(tmap_path)
    tmap_data = tmap_img.get_fdata()

    # Apply threshold
    thresholded_data = tmap_data > threshold

    # Label clusters
    labeled_data, num_clusters = label(thresholded_data)
    print(f"Found {num_clusters} clusters (before extent threshold).")

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save each cluster that meets the extent threshold
    cluster_paths = []
    retained_clusters = 0
    for cluster_id in range(1, num_clusters + 1):
        cluster_mask = (labeled_data == cluster_id).astype(np.int16)
        # Count the number of voxels in the cluster
        voxel_count = np.sum(cluster_mask)

        # Skip clusters that do not meet the extent threshold
        if voxel_count < extent_threshold:
            continue

        # Save the cluster mask
        cluster_img = nib.Nifti1Image(cluster_mask, affine=tmap_img.affine, header=tmap_img.header)
        output_path = os.path.join(output_dir, f"cluster_{retained_clusters + 1}.nii")
        nib.save(cluster_img, output_path)
        cluster_paths.append(output_path)
        retained_clusters += 1
        print(f"Saved cluster {retained_clusters} with {voxel_count} voxels to {output_path}")

    print(f"Retained {retained_clusters} clusters (after extent threshold).")

    # Return cluster paths and the affine matrix
    return cluster_paths, tmap_img.affine


def calculate_centroids_and_peaks(cluster_paths, tmap_path):
    """
    Calculates the centroid and peak coordinates of each cluster.

    Parameters:
        cluster_paths (list): List of paths to cluster NIfTI masks.
        tmap_path (str): Path to the original T-map NIfTI file.

    Returns:
        list: A list of dictionaries for each cluster containing centroid and peak coordinates.
    """
    # Load the T-map
    tmap_img = nib.load(tmap_path)
    tmap_data = tmap_img.get_fdata()

    results = []
    for cluster_path in cluster_paths:
        # Load the cluster mask
        cluster_img = nib.load(cluster_path)
        cluster_data = cluster_img.get_fdata()

        # Mask the T-map with the cluster mask
        weighted_data = cluster_data * tmap_data

        # Find voxel coordinates
        coords = np.array(np.nonzero(cluster_data))
        weights = weighted_data[cluster_data > 0]

        # Calculate the weighted centroid
        if weights.sum() > 0:
            centroid = np.average(coords, axis=1, weights=weights)
            # Transform centroid to world coordinates
            centroid_world = nib.affines.apply_affine(tmap_img.affine, centroid)
        else:
            centroid_world = (np.nan, np.nan, np.nan)  # In case of no weights

        # Find the peak coordinate (voxel with the highest T-statistic)
        peak_voxel_index = np.unravel_index(np.argmax(weighted_data, axis=None), cluster_data.shape)
        peak_world = nib.affines.apply_affine(tmap_img.affine, peak_voxel_index)

        # Store results
        results.append({
            "centroid": tuple(centroid_world),
            "peak": tuple(peak_world)
        })
    
    return results

# Updated to include peak T value


In [7]:
# thresholded_binary = '/Users/neel/Desktop/SPM-V_II_Zipf_multireg (FINAL)/VisualWord_roi.nii'
roi_output_dir = '/home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/roi'
if not os.path.exists(roi_output_dir):
    os.makedirs(roi_output_dir)
tmap = str(Path(roi_output_dir).parent / 'spmT_0001.nii')
print(tmap)
save_path = str(Path(roi_output_dir).parent / 'cluster_centroids.csv')
print(save_path)

/home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/spmT_0001.nii
/home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/cluster_centroids.csv


In [8]:
#delete all the contents of the roi_output_dir before running the code
for file in os.listdir(roi_output_dir):
    os.remove(os.path.join(roi_output_dir, file))
cluster_paths, affine = separate_clusters(tmap, roi_output_dir, 7.41,extent_threshold=20)
results = calculate_centroids_and_peaks(cluster_paths, tmap)


Found 8 clusters (before extent threshold).
Saved cluster 1 with 214 voxels to /home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/roi/cluster_1.nii
Saved cluster 2 with 186 voxels to /home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/roi/cluster_2.nii
Saved cluster 3 with 97 voxels to /home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/roi/cluster_3.nii
Saved cluster 4 with 416 voxels to /home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/roi/cluster_4.nii
Saved cluster 5 with 63 voxels to /home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/roi/cluster_5.nii
Retained 5 clusters (after extent threshold).


In [9]:

# Calculate peak T-values and store results
sorted_results = []
for i, result in enumerate(results, start=1):
    centroid = [round(coord) for coord in result["centroid"]]
    peak = [round(coord) for coord in result["peak"]]
    
    # Load the T-map to get the peak T-value
    tmap_img = nib.load(tmap)
    tmap_data = tmap_img.get_fdata()
    
    # Convert the peak coordinate from world space to voxel space
    peak_voxel = np.round(nib.affines.apply_affine(np.linalg.inv(tmap_img.affine), result["peak"])).astype(int)
    
    # Ensure the voxel indices are within bounds
    if (0 <= peak_voxel[0] < tmap_data.shape[0] and
        0 <= peak_voxel[1] < tmap_data.shape[1] and
        0 <= peak_voxel[2] < tmap_data.shape[2]):
        peak_t_value = tmap_data[tuple(peak_voxel)]
    else:
        peak_t_value = float('nan')  # Out of bounds, assign NaN

    # Calculate cluster size in voxels
    cluster_path = cluster_paths[i - 1]
    cluster_img = nib.load(cluster_path)
    cluster_data = cluster_img.get_fdata()
    cluster_size = int(np.sum(cluster_data > 0))  # Count non-zero voxels

    sorted_results.append({
        'Cluster #': i,
        'Centroid': centroid,
        'Peak': peak,
        'Peak T-value': peak_t_value,
        'Extent': cluster_size
    })

# Sort results by peak T-value in descending order
sorted_results.sort(key=lambda x: x['Peak T-value'], reverse=True)

# Reassign cluster numbers in order
for i, result in enumerate(sorted_results, start=1):
    result['Cluster #'] = i

# Print sorted results
for result in sorted_results:
    print(f"Cluster {result['Cluster #']}:")
    print(f"  Centroid = {result['Centroid']}")
    print(f"  Peak = {result['Peak']}")
    print(f"  Peak T-value = {round(result['Peak T-value'], 2)}")
    print(f"  Extent = {result['Extent']} voxels")

# Save sorted results to a CSV file
with open(save_path, 'w', newline='') as csvfile:
    fieldnames = ['Cluster #', 'Centroid', 'Peak', 'Peak T-value', 'Extent']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in sorted_results:
        writer.writerow({
            'Cluster #': result['Cluster #'],
            'Centroid': result['Centroid'],
            'Peak': result['Peak'],
            'Peak T-value': round(result['Peak T-value'], 2),
            'Extent': result['Extent']
        })

Cluster 1:
  Centroid = [-24, -92, -9]
  Peak = [-18, -94, -8]
  Peak T-value = 12.8
  Extent = 416 voxels
Cluster 2:
  Centroid = [-55, -9, -10]
  Peak = [-58, -10, -8]
  Peak T-value = 10.79
  Extent = 186 voxels
Cluster 3:
  Centroid = [-54, -36, 3]
  Peak = [-50, -38, 2]
  Peak T-value = 10.04
  Extent = 214 voxels
Cluster 4:
  Centroid = [-49, 32, 1]
  Peak = [-46, 32, -2]
  Peak T-value = 9.15
  Extent = 97 voxels
Cluster 5:
  Centroid = [24, -93, -4]
  Peak = [24, -96, -2]
  Peak T-value = 8.83
  Extent = 63 voxels
